In [1]:
import banglanltk as bn
from bltk.langtools import Tokenizer
from bltk.langtools import remove_stopwords
import BnLemma as lm
import numpy as np
import json
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Activation,Dense
from tensorflow.keras.optimizers import SGD

C:\Users\ASUS\anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
tokenizer = Tokenizer()
words = []
classes = []
documents = []

data_file =open('bangla.json',encoding='utf-8').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent["patterns"]:

        # tokenize each word
        sent=bn.clean_text(pattern)
        w =tokenizer.word_tokenizer(sent)
        #print(w)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
documents

[(['হাই', 'আছে'], 'শুভেচ্ছা'),
 (['আপনি', 'কেমন', 'আছেন'], 'শুভেচ্ছা'),
 (['সেখানে', 'কেউ', 'আছেন'], 'শুভেচ্ছা'),
 (['আরে'], 'শুভেচ্ছা'),
 (['হোলা'], 'শুভেচ্ছা'),
 (['হ্যালো'], 'শুভেচ্ছা'),
 (['শুভ', 'দিন'], 'শুভেচ্ছা'),
 (['বাই'], 'বিদায়'),
 (['পরে', 'দেখা', 'হবে'], 'বিদায়'),
 (['বিদায়'], 'বিদায়'),
 (['আপনার', 'সাথে', 'আড্ডা', 'দিয়ে', 'ভালো', 'লাগলো', 'বাই'], 'বিদায়'),
 (['পরবর্তী', 'সময়', 'পর্যন্ত'], 'বিদায়'),
 (['ধন্যবাদ'], 'ধন্যবাদ'),
 (['ধন্যবাদ'], 'ধন্যবাদ'),
 (['এটি', 'সহায়ক'], 'ধন্যবাদ'),
 (['আন্তরিক', 'ধন্যবাদ'], 'ধন্যবাদ'),
 (['আমাকে', 'সাহায্য', 'করার', 'জন্য', 'ধন্যবাদ'], 'ধন্যবাদ'),
 (['আপনি', 'কিভাবে', 'আমাকে', 'সাহায্য', 'করতে', 'পারেন'], 'বিকল্প'),
 (['তুমি', 'কি', 'করতে', 'পার'], 'বিকল্প'),
 (['আপনি', 'কি', 'সাহায্য', 'প্রদান', 'করেন'], 'বিকল্প'),
 (['আপনি', 'কিভাবে', 'সহায়ক', 'হতে', 'পারেন'], 'বিকল্প'),
 (['কি', 'সমর্থন', 'দেওয়া', 'হয়'], 'বিকল্প'),
 (['পদার্থবিদ্যা', 'কি'], 'সংজ্ঞা'),
 (['পদার্থবিদ্যা', 'মানে'], 'সংজ্ঞা'),
 (['পদার্থবিজ্ঞানের', 'সংজ্ঞা', '

In [4]:
words

['হাই',
 'আছে',
 'আপনি',
 'কেমন',
 'আছেন',
 'সেখানে',
 'কেউ',
 'আছেন',
 'আরে',
 'হোলা',
 'হ্যালো',
 'শুভ',
 'দিন',
 'বাই',
 'পরে',
 'দেখা',
 'হবে',
 'বিদায়',
 'আপনার',
 'সাথে',
 'আড্ডা',
 'দিয়ে',
 'ভালো',
 'লাগলো',
 'বাই',
 'পরবর্তী',
 'সময়',
 'পর্যন্ত',
 'ধন্যবাদ',
 'ধন্যবাদ',
 'এটি',
 'সহায়ক',
 'আন্তরিক',
 'ধন্যবাদ',
 'আমাকে',
 'সাহায্য',
 'করার',
 'জন্য',
 'ধন্যবাদ',
 'আপনি',
 'কিভাবে',
 'আমাকে',
 'সাহায্য',
 'করতে',
 'পারেন',
 'তুমি',
 'কি',
 'করতে',
 'পার',
 'আপনি',
 'কি',
 'সাহায্য',
 'প্রদান',
 'করেন',
 'আপনি',
 'কিভাবে',
 'সহায়ক',
 'হতে',
 'পারেন',
 'কি',
 'সমর্থন',
 'দেওয়া',
 'হয়',
 'পদার্থবিদ্যা',
 'কি',
 'পদার্থবিদ্যা',
 'মানে',
 'পদার্থবিজ্ঞানের',
 'সংজ্ঞা',
 'কি',
 'আমাকে',
 'পদার্থবিজ্ঞানের',
 'মৌলিক',
 'ধারণা',
 'বল',
 'আমাকে',
 'পদার্থবিদ্যা',
 'সম্পর্কে',
 'বলুন',
 'আমাকে',
 'পদার্থবিজ্ঞানের',
 'কিছু',
 'বাস্তব',
 'জীবনের',
 'উদাহরণ',
 'বলুন',
 'পদার্থবিজ্ঞানের',
 'কিছু',
 'উদাহরণ',
 'আমাকে',
 'পদার্থবিজ্ঞানের',
 'কিছু',
 'দৃশ্য',
 'বলুন',
 'পদার্থবিদ্যা',
 'এবং

In [5]:
classes

['শুভেচ্ছা',
 'বিদায়',
 'ধন্যবাদ',
 'বিকল্প',
 'সংজ্ঞা',
 'উদাহরণ',
 'গণিত',
 'রসায়ন',
 'জীববিদ্যা',
 'চিকিৎসা বিজ্ঞান',
 'মৌলিক',
 'যৌগিক',
 'বিজ্ঞানী',
 'ভেক্টর',
 'মাত্রা',
 'কোণ',
 'অনন্য',
 'বিন্দু',
 'বিশ্রাম',
 'গতি',
 'আত্মীয়',
 'স্থানচ্যুতি',
 'স্থানচ্যুতি_আইন',
 'ত্বরণ',
 'ত্বরণ_আইন',
 'গ্যালিলিও_ল',
 'সর্বোচ্চ_উচ্চতা',
 'বল',
 'মৌলিক_শক্তি',
 'মাধ্যাকর্ষণ_বল',
 'ইলেক্ট্রোম্যাগনেটিক_ফোর্স',
 'শক্তিশালী_পরমাণু',
 'দুর্বল_পরমাণু',
 'নিউটন_লজ',
 'ঘূর্ণন_গতি',
 'কৌণিক_সরন',
 'কৌণিক_বেগ',
 'কৌণিক_ত্বরণ',
 'মুহূর্ত_জড়তা',
 'ব্যাসার্ধ_গাইরেশন',
 'কেন্দ্রীয়_বল',
 'কৌণিক_ভরবেগ',
 'মাধ্যাকর্ষণ',
 'মাধ্যাকর্ষণ_নিউটন',
 'কেপলার_ল',
 'অভিকর্ষ',
 'ত্বরণ_অভিকর্ষ',
 'ধ্রুবক_মাধ্যাকর্ষণ',
 'variation_g',
 'পৃথিবীর_ঘনত্ব',
 ' কৃত্রিম_কেন্দ্রমুখী',
 'কৃত্রিম_অভিকর্ষ',
 'কৃত্রিম_বেগ',
 'কৃত্রিম_উচ্চতা',
 'মহাকর্ষীয়_প্রাবল্য',
 'মহাকর্ষীয়_বিভব',
 'বিন্দু_বিভব',
 'পৃথিবীতে_মুক্তিবেগ',
 'সুত্র_কাজ',
 'আহ্নিক',
 'কাজ',
 'স্প্রিং_কাজ',
 'মহাকর্ষ_কৃতকাজ',
 'গতিশক্তি_সুত্র',
 'শক্তি',
 'সুত্র_বিভব

In [6]:
len(classes) 

576

In [7]:
words= remove_stopwords(words)

In [8]:
words

['হাই',
 'কেমন',
 'আছেন',
 'কেউ',
 'আছেন',
 'আরে',
 'হোলা',
 'হ্যালো',
 'শুভ',
 'দিন',
 'বাই',
 'দেখা',
 'বিদায়',
 'সাথে',
 'আড্ডা',
 'দিয়ে',
 'ভালো',
 'লাগলো',
 'বাই',
 'পরবর্তী',
 'পর্যন্ত',
 'ধন্যবাদ',
 'ধন্যবাদ',
 'সহায়ক',
 'আন্তরিক',
 'ধন্যবাদ',
 'সাহায্য',
 'করার',
 'ধন্যবাদ',
 'সাহায্য',
 'করতে',
 'তুমি',
 'করতে',
 'পার',
 'সাহায্য',
 'প্রদান',
 'করেন',
 'সহায়ক',
 'সমর্থন',
 'দেওয়া',
 'পদার্থবিদ্যা',
 'পদার্থবিদ্যা',
 'মানে',
 'পদার্থবিজ্ঞানের',
 'সংজ্ঞা',
 'পদার্থবিজ্ঞানের',
 'মৌলিক',
 'ধারণা',
 'বল',
 'পদার্থবিদ্যা',
 'বলুন',
 'পদার্থবিজ্ঞানের',
 'বাস্তব',
 'জীবনের',
 'উদাহরণ',
 'বলুন',
 'পদার্থবিজ্ঞানের',
 'উদাহরণ',
 'পদার্থবিজ্ঞানের',
 'দৃশ্য',
 'বলুন',
 'পদার্থবিদ্যা',
 'গণিতের',
 'সম্পর্ক',
 'পদার্থবিদ্যা',
 'গণিতের',
 'সম্পর্ক',
 'বলুন',
 'পদার্থবিদ্যা',
 'গণিতের',
 'প্রকৃত',
 'সংযোগ',
 'দয়া',
 'পদার্থবিদ্যা',
 'গণিতের',
 'সংযোগ',
 'বর্ণনা',
 'করুন',
 'পদার্থবিদ্যা',
 'রসায়নের',
 'সম্পর্ক',
 'পদার্থবিদ্যা',
 'রসায়নের',
 'সম্পর্ক',
 'বলুন',
 'পদার্থবিদ্যা',
 'রসায়

In [9]:
bl = lm.Lemmatizer() 

In [10]:
words.remove('N')

In [11]:
words.remove('বযায়')

In [12]:
words.remove('বয়েলের')

In [13]:
words.remove('বয়েলের')

In [14]:
words.remove('বয়েলের')

In [15]:
bl = lm.Lemmatizer()
words = [bl.lemma(w) for w in words]

In [16]:
len(words)

7899

In [17]:
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

2272 documents
576 classes ['', ' কুলম্বের_সংজ্ঞা', ' কুলম্বের_সূত্রটি', ' কৃত্রিম_কেন্দ্রমুখী', ' কৈশিক_আয়তন', ' ক্যালভিন', ' ত্বরণ', ' তড়িৎ_বল', ' তড়িৎচালক_শক্তি', ' দূরত্ব', ' বায়ুশক্তি', ' ভরবেগের_সংরক্ষণ', ' শ্রেণি_বর্তনী', ' সান্দ্রতা', ' হুকের_সূত্র', '.তড়িৎ_প্রাবল্য,', '.তড়িৎ_বিভব,', 'Cp_Cv', 'n_টাইপ ', 'p_টাইপ ', 'variation_g', 'অক্ষ_উপপাদ্য', 'অক্ষের_জড়তার', 'অগ্রগামী', 'অগ্রগামী_তরঙ্গ', 'অতি_পরিবাহিতা', 'অনন্য', 'অনবায়নযোগ্য_শক্তি', 'অনিয়মিত', 'অনুদৈর্ঘ্য_তরঙ্গ', 'অনুনাদ', 'অনুপ্রস্থ_তরঙ্গ ', 'অনুবন্ধী_ফোকাস', 'অনুভূমিক_পাল্লা', 'অন্তবেগ', 'অন্তস্থ_শক্তি', 'অপবর্তন', 'অপরিবাহক', 'অপরিবাহী', 'অপ্রত্যাবর্তী_প্রক্রিয়াা', 'অবস্থান_ভেক্টর', 'অভিকর্ষ', 'অভিকর্ষ_কেন্দ্র ', 'অভ্যন্তরীণ_রোধ ', 'অভ্যন্তরীণ_শক্তি', 'অর্কেস্ট্রা', 'অর্ধপরিবাহী', 'অর্ধায়ু', 'অশ্বক্ষমতা', 'অষ্টক', 'অসম_ত্বরণ', 'অসম_দ্রুতি', 'অসম_বেগ', 'অসম্পৃক্ত_বাষ্প', 'অসহ_পীড়ন', 'অসাম্য_বল', 'অস্পর্শ_বল', 'অ্যামিটার', 'অ্যাম্পিয়ার', 'অ্যাম্পিয়ারের', 'আড়_তরঙ্গ', 'আত্মীয়', 'আদর্শ_গতিশক্তি', 'আদর্শ_গ্যাস', 'আদর্শ_গ

In [18]:
len(words)

622

In [19]:
len(words)

622

In [20]:
len(classes)

576

In [32]:

import random
pickle.dump(words, open('words43.pkl', 'wb'))
pickle.dump(classes, open('classes43.pkl', 'wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
   
  
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [bl.lemma(word) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model43', hist)

print("model created and saved")

C:\Users\ASUS\AppData\Local\Temp/ipykernel_11660/111638855.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Training data created


C:\Users\ASUS\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
455/455 [==============================] - 1s 935us/step - loss: 6.3557 - accuracy: 0.0092
Epoch 2/200
455/455 [==============================] - 0s 983us/step - loss: 6.3254 - accuracy: 0.0123
Epoch 3/200
455/455 [==============================] - 1s 2ms/step - loss: 6.2719 - accuracy: 0.0128
Epoch 4/200
455/455 [==============================] - 1s 1ms/step - loss: 6.1755 - accuracy: 0.0189
Epoch 5/200
455/455 [==============================] - 0s 904us/step - loss: 6.0132 - accuracy: 0.0264
Epoch 6/200
455/455 [==============================] - 0s 838us/step - loss: 5.8053 - accuracy: 0.0343
Epoch 7/200
455/455 [==============================] - 0s 885us/step - loss: 5.5184 - accuracy: 0.0502
Epoch 8/200
455/455 [==============================] - 1s 2ms/step - loss: 5.2629 - accuracy: 0.0581
Epoch 9/200
455/455 [==============================] - 1s 1ms/step - loss: 4.8993 - accuracy: 0.0845
Epoch 10/200
455/455 [==============================] - 1s 2ms/step - loss: 4.643

455/455 [==============================] - 0s 837us/step - loss: 2.1008 - accuracy: 0.4635
Epoch 81/200
455/455 [==============================] - 0s 865us/step - loss: 2.0101 - accuracy: 0.4776
Epoch 82/200
455/455 [==============================] - 0s 958us/step - loss: 2.0253 - accuracy: 0.4670
Epoch 83/200
455/455 [==============================] - 0s 907us/step - loss: 2.0053 - accuracy: 0.4643
Epoch 84/200
455/455 [==============================] - 0s 861us/step - loss: 2.0285 - accuracy: 0.4798
Epoch 85/200
455/455 [==============================] - 0s 881us/step - loss: 1.9981 - accuracy: 0.4811
Epoch 86/200
455/455 [==============================] - 0s 884us/step - loss: 1.9854 - accuracy: 0.4850
Epoch 87/200
455/455 [==============================] - 0s 840us/step - loss: 1.9886 - accuracy: 0.4820
Epoch 88/200
455/455 [==============================] - 0s 898us/step - loss: 2.0196 - accuracy: 0.4727
Epoch 89/200
455/455 [==============================] - 1s 1ms/step - loss: 1

455/455 [==============================] - 0s 1ms/step - loss: 2.0043 - accuracy: 0.5013
Epoch 159/200
455/455 [==============================] - 0s 856us/step - loss: 1.9758 - accuracy: 0.5101
Epoch 160/200
455/455 [==============================] - 0s 1ms/step - loss: 2.0176 - accuracy: 0.4899
Epoch 161/200
455/455 [==============================] - 0s 984us/step - loss: 1.9965 - accuracy: 0.5110
Epoch 162/200
455/455 [==============================] - 0s 898us/step - loss: 2.1028 - accuracy: 0.4947
Epoch 163/200
455/455 [==============================] - 0s 856us/step - loss: 1.9942 - accuracy: 0.5018
Epoch 164/200
455/455 [==============================] - 0s 887us/step - loss: 2.0437 - accuracy: 0.4877
Epoch 165/200
455/455 [==============================] - 0s 834us/step - loss: 2.0809 - accuracy: 0.4969
Epoch 166/200
455/455 [==============================] - 0s 835us/step - loss: 1.9809 - accuracy: 0.5167
Epoch 167/200
455/455 [==============================] - 0s 849us/step - 

In [22]:
pattern_words.remove('N')

In [24]:
pattern_words.remove('বযায়')

In [26]:
pattern_words.remove('বয়েলের')

In [28]:
pattern_words.remove('বয়েলের')

In [31]:
pattern_words.remove('বয়েলের')

In [33]:
import banglanltk as bn
from bltk.langtools import Tokenizer
from bltk.langtools import remove_stopwords
import BnLemma as lm
import numpy as np
import json
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Activation,Dense
from tensorflow.keras.optimizers import SGD
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model43')
import json
import random
import pyttsx3  as pt


intents = json.loads(open('bangla.json',encoding='utf-8').read())
words = pickle.load(open('words43.pkl','rb'))
classes = pickle.load(open('classes43.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sent=bn.clean_text(sentence)
    w =tokenizer.word_tokenizer(sent)
    # stem each word - create short form for word
    sentence_words = [bl.lemma(word) for word in w]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Javed: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Physic Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#25cdf7", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


1/1 [==============================] - 0s 74ms/step
